# Sunbird RC API - Water Facility Demo (Public URL)

This notebook demonstrates how to interact with Sunbird RC Registry API for Water Facilities
deployed at https://sunbird-rc.akvotest.org.

## Prerequisites
- Sunbird RC services deployed at https://sunbird-rc.akvotest.org
- WaterFacility schema configured
- Keycloak `admin-api` client secret (from k8s secret `sunbird-rc`)

## Setup

In [1]:
import requests
import json
import os
import pandas as pd
from getpass import getpass
from datetime import datetime
from IPython.display import display, Markdown, JSON

# Configuration
DOMAIN = "https://sunbird-rc.akvotest.org"
BASE_URL = f"{DOMAIN}/api/v1"
KEYCLOAK_URL = f"{DOMAIN}/auth/realms/sunbird-rc/protocol/openid-connect/token"

# Auth credentials (demo-api client)
CLIENT_ID = "demo-api"
CLIENT_SECRET = os.environ.get("SUNBIRD_DEMO_API_CLIENT_SECRET") or getpass("Enter demo-api client secret: ")

print(f"Registry API: {BASE_URL}")
print(f"Keycloak Token URL: {KEYCLOAK_URL}")

Registry API: https://sunbird-rc.akvotest.org/api/v1
Keycloak Token URL: https://sunbird-rc.akvotest.org/auth/realms/sunbird-rc/protocol/openid-connect/token


## 0. Authenticate with Keycloak

Obtain an access token using the `admin-api` client credentials.

In [2]:
token_response = requests.post(KEYCLOAK_URL, data={
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "grant_type": "client_credentials"
})

if token_response.status_code == 200:
    ACCESS_TOKEN = token_response.json()["access_token"]
    AUTH_HEADERS = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {ACCESS_TOKEN}"
    }
    print(f"Token obtained (expires in {token_response.json()['expires_in']}s)")
else:
    print(f"Auth failed: {token_response.status_code}")
    print(token_response.text)

Token obtained (expires in 300s)


## 1. Check Registry Health

In [3]:
response = requests.get(f"{DOMAIN}/health")
health = response.json()

print(f"Status: {health['result']['healthy']}")
print(f"Service: {health['result']['name']}")
display(JSON(health, expanded=True))

Status: True
Service: sunbirdrc-registry-api


<IPython.core.display.JSON object>

## 2. List All Water Facilities

In [ ]:
response = requests.get(f"{BASE_URL}/WaterFacility", headers=AUTH_HEADERS)
data = response.json()

print(f"Total Water Facilities: {data['totalCount']}\n")

if data['totalCount'] > 0:
    df = pd.DataFrame(data['data'])
    desired_columns = ['geoCode', 'wfId', 'waterPointType', 'extractionType', 'pumpType', 'owner', 'osid']
    columns = [col for col in desired_columns if col in df.columns]
    display(df[columns])

    if 'location' in df.columns:
        print("\nLocation Details:")
        location_df = pd.json_normalize(df['location'])
        location_df['geoCode'] = df['geoCode'].values
        display(location_df[['geoCode', 'county', 'district', 'community']])
else:
    print("No water facilities found")

## 3. Create a New Water Facility

In [ ]:
new_facility = {
    "geoCode": "test001",
    "waterPointType": "Protected dug well",
    "location": {
        "county": "Bong",
        "district": "Kpaai",
        "community": "Test Community",
        "coordinates": {
            "lat": 6.99428667,
            "lon": -9.46091172,
            "elevation": 286.4
        }
    },
    "extractionType": "Manual",
    "pumpType": "Afridev",
    "hasDepthInfo": True,
    "depthMetres": 15.0,
    "installer": "NGO",
    "owner": "Community",
    "funder": "USAID",
    "photoUrl": "https://example.com/photo.jpg"
}

response = requests.post(
    f"{BASE_URL}/WaterFacility",
    headers=AUTH_HEADERS,
    json=new_facility
)

if response.status_code == 200:
    result = response.json()
    osid = result['result']['WaterFacility']['osid']
    print("Water Facility created successfully!")
    print(f"osid: {osid}")

    fetch_response = requests.get(f"{BASE_URL}/WaterFacility/{osid}", headers=AUTH_HEADERS)
    if fetch_response.status_code == 200:
        created_facility = fetch_response.json()
        print(f"wfId: {created_facility.get('wfId', 'N/A')}")
        print(f"\nGenerated wfId format: WF-<COUNTY>-<DISTRICT>-<TYPE>-<HASH>")
        display(JSON(created_facility, expanded=True))
else:
    print(f"Error: {response.status_code}")
    print(response.text)

## 4. Get a Specific Water Facility by ID

In [6]:
response = requests.get(f"{BASE_URL}/WaterFacility", headers=AUTH_HEADERS)
facilities = response.json()['data']

if facilities:
    facility_id = facilities[0]['osid']
    print(f"Fetching water facility: {facility_id}\n")
    response = requests.get(f"{BASE_URL}/WaterFacility/{facility_id}", headers=AUTH_HEADERS)
    facility = response.json()
    display(JSON(facility, expanded=True))
else:
    print("No facilities found")

Fetching water facility: 1-90ba7d17-ccef-4331-9e9d-cd910eed15d8



<IPython.core.display.JSON object>

## 5. Update a Water Facility

In [ ]:
response = requests.get(f"{BASE_URL}/WaterFacility", headers=AUTH_HEADERS)
facilities = response.json()['data']

if facilities:
    facility_id = facilities[0]['osid']

    update_data = facilities[0].copy()
    update_data['depthMetres'] = 25.0
    update_data['funder'] = 'Updated Funder Organization'

    response = requests.put(
        f"{BASE_URL}/WaterFacility/{facility_id}",
        headers=AUTH_HEADERS,
        json=update_data
    )

    if response.status_code == 200:
        print("Water Facility updated successfully!")
        display(JSON(response.json(), expanded=True))
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
else:
    print("No facilities available to update")

## 6. Search Water Facilities

In [ ]:
search_query = {
    "filters": {
        "waterPointType": {"eq": "Protected dug well"}
    }
}

response = requests.post(
    f"{BASE_URL}/WaterFacility/search",
    headers=AUTH_HEADERS,
    json=search_query
)

if response.status_code == 200:
    results = response.json()
    print(f"Found {results['totalCount']} Protected dug wells\n")

    if results['totalCount'] > 0:
        df = pd.DataFrame(results['data'])
        desired_columns = ['wfId', 'geoCode', 'waterPointType', 'extractionType', 'pumpType', 'owner']
        columns = [col for col in desired_columns if col in df.columns]
        display(df[columns])

        if 'location' in df.columns:
            print("\nLocation Details:")
            location_df = pd.json_normalize(df['location'])
            location_df['geoCode'] = df['geoCode'].values
            if 'wfId' in df.columns:
                location_df['wfId'] = df['wfId'].values
                display(location_df[['wfId', 'geoCode', 'county', 'district', 'community']])
            else:
                display(location_df[['geoCode', 'county', 'district', 'community']])
    else:
        print("No results found")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

## 7. Bulk Create Water Facilities

In [ ]:
# Sample bulk data with Liberia water points
bulk_facilities = [
    {
        "geoCode": "w4q6yvl",
        "waterPointType": "Tube well or borehole",
        "location": {
            "county": "Nimba",
            "district": "Buu-Yao",
            "community": "Say's",
            "coordinates": {"lat": 6.99428667, "lon": -9.46091172, "elevation": 286.4}
        },
        "extractionType": "Manual",
        "pumpType": "India Mark",
        "hasDepthInfo": False,
        "installer": "Government",
        "owner": "Community",
        "funder": "LWC",
        "photoUrl": "https://flowliberia.s3.amazonaws.com/images/cfd2e566-18ee-4952-b08b-31db5fef24bf.jpg"
    },
    {
        "geoCode": "w4rdu39",
        "waterPointType": "Tube well or borehole",
        "location": {
            "county": "Bong",
            "district": "Kpaai",
            "community": "Siawolor Town",
            "coordinates": {"lat": 6.99448282, "lon": -9.46136413, "elevation": 290.7}
        },
        "extractionType": "Electrical",
        "pumpType": "Kardia",
        "hasDepthInfo": True,
        "depthMetres": 108.0,
        "installer": "NGO",
        "owner": "Health Facility",
        "funder": "USAID",
        "photoUrl": "https://flowliberia.s3.amazonaws.com/images/845285a9-6a39-44a7-a259-9327db56a5a0.jpg"
    },
    {
        "geoCode": "w4qdecx",
        "waterPointType": "Protected dug well",
        "location": {
            "county": "Lofa",
            "district": "Zorzor",
            "community": "Zoe",
            "coordinates": {"lat": 6.99431151, "lon": -9.46096315, "elevation": 279.7}
        },
        "extractionType": "Manual",
        "pumpType": "Afridev",
        "hasDepthInfo": False,
        "installer": "NGO",
        "owner": "Health Facility",
        "funder": "LWC",
        "photoUrl": "https://flowliberia.s3.amazonaws.com/images/591f7c89-7dc3-44af-a684-e4ab8e82a0c6.jpg"
    },
    {
        "geoCode": "rugw4rt",
        "waterPointType": "Rainwater (harvesting)",
        "location": {
            "county": "Grand Gedeh",
            "district": "Tchien",
            "community": "Zwedru",
            "coordinates": {"lat": 6.061545, "lon": -8.13854, "elevation": 254.8}
        },
        "numTaps": 1.0,
        "hasDepthInfo": False,
        "installer": "Government",
        "owner": "Community",
        "funder": "Unicef",
        "photoUrl": "https://flowliberia.s3.amazonaws.com/images/668b7ad7-4561-4850-90bd-8c7ff7dab977.jpg"
    },
    {
        "geoCode": "w4t1uky",
        "waterPointType": "Protected spring",
        "location": {
            "county": "Bong",
            "district": "Boinsen",
            "community": "Samai",
            "coordinates": {"lat": 6.99476405, "lon": -9.46100939, "elevation": 293.1}
        },
        "installer": "Private",
        "owner": "Private Individual",
        "funder": "Private",
        "photoUrl": "https://flowliberia.s3.amazonaws.com/images/dbd8c589-b69a-4794-a14b-48672812920a.jpg"
    }
]

print(f"Creating {len(bulk_facilities)} water facilities...\n")

results = []
for i, facility in enumerate(bulk_facilities, 1):
    response = requests.post(
        f"{BASE_URL}/WaterFacility",
        headers=AUTH_HEADERS,
        json=facility
    )

    if response.status_code == 200:
        result = response.json()
        osid = result['result']['WaterFacility']['osid']

        fetch_response = requests.get(f"{BASE_URL}/WaterFacility/{osid}", headers=AUTH_HEADERS)
        wfId = "N/A"
        if fetch_response.status_code == 200:
            created = fetch_response.json()
            wfId = created.get('wfId', 'N/A')

        print(f"{i}. {facility['geoCode']} ({facility['waterPointType']})")
        print(f"   wfId: {wfId}")
        print(f"   osid: {osid}\n")
        results.append({'status': 'success', 'osid': osid, 'wfId': wfId, 'geoCode': facility['geoCode']})
    else:
        print(f"{i}. {facility['geoCode']} - FAILED ({response.status_code})")
        print(f"   Error: {response.text}\n")
        results.append({'status': 'failed', 'geoCode': facility['geoCode'], 'error': response.text})

print(f"{'='*50}")
print(f"Created {len([r for r in results if r['status'] == 'success'])} facilities")
print(f"Failed {len([r for r in results if r['status'] == 'failed'])} facilities")

## 7.1 Test Duplicate Rejection (wfId uniqueness)

The wfId is generated based on a hash of facility attributes (geoCode, waterPointType, location).
Attempting to create a facility with the same attributes should be rejected due to the unique constraint on wfId.

In [ ]:
duplicate_facility = {
    "geoCode": "w4q6yvl",  # Same geoCode as bulk create
    "waterPointType": "Tube well or borehole",  # Same type
    "location": {
        "county": "Nimba",  # Same location
        "district": "Buu-Yao",
        "community": "Say's",
        "coordinates": {"lat": 6.99428667, "lon": -9.46091172, "elevation": 286.4}
    },
    "extractionType": "Electrical",  # Different extraction - should not matter
    "pumpType": "Afridev",  # Different pump - should not matter
    "installer": "Private",  # Different installer - should not matter
    "owner": "Private Individual",  # Different owner - should not matter
    "funder": "Different Funder"  # Different funder - should not matter
}

print("Testing duplicate rejection...")
print(f"Attempting to create: {duplicate_facility['geoCode']}")
print(f"Location: {duplicate_facility['location']['county']}, {duplicate_facility['location']['district']}")
print()

response = requests.post(
    f"{BASE_URL}/WaterFacility",
    headers=AUTH_HEADERS,
    json=duplicate_facility
)

if response.status_code == 200:
    result = response.json()
    osid = result['result']['WaterFacility']['osid']
    print(f"Unexpected: Facility was created with osid: {osid}")

    fetch_response = requests.get(f"{BASE_URL}/WaterFacility/{osid}", headers=AUTH_HEADERS)
    if fetch_response.status_code == 200:
        created = fetch_response.json()
        print(f"wfId: {created.get('wfId', 'N/A')}")
else:
    print(f"Expected behavior: Duplicate rejected!")
    print(f"Status code: {response.status_code}")
    try:
        error_detail = response.json()
        print(f"Error: {json.dumps(error_detail, indent=2)}")
    except:
        print(f"Error: {response.text}")

## 8. Export to CSV

In [ ]:
response = requests.get(f"{BASE_URL}/WaterFacility", headers=AUTH_HEADERS)
data = response.json()

if data['totalCount'] > 0:
    df = pd.DataFrame(data['data'])

    if 'location' in df.columns:
        location_df = pd.json_normalize(df['location'])
        location_df.columns = [f'location_{col}' for col in location_df.columns]
        df = pd.concat([df.drop('location', axis=1), location_df], axis=1)

    export_columns = [
        'geoCode', 'wfId', 'waterPointType', 'extractionType', 'pumpType',
        'location_county', 'location_district', 'location_community',
        'location_coordinates.lat', 'location_coordinates.lon', 'location_coordinates.elevation',
        'numTaps', 'hasDepthInfo', 'depthMetres',
        'installer', 'owner', 'funder', 'photoUrl',
        'osid', 'osCreatedAt'
    ]
    available_columns = [col for col in export_columns if col in df.columns]

    filename = f'water_facilities_export.csv'
    df[available_columns].to_csv(filename, index=False)

    print(f"Exported {len(df)} water facilities to: {filename}")
    display(df[available_columns].head())
else:
    print("No water facilities to export")

## 9. Delete All Water Facilities

In [ ]:
response = requests.get(f"{BASE_URL}/WaterFacility", headers=AUTH_HEADERS)
facilities = response.json()['data']

if facilities:
    print(f"Deleting {len(facilities)} water facilities...\n")

    deleted = 0
    failed = 0
    for facility in facilities:
        osid = facility['osid']
        geoCode = facility.get('geoCode', 'Unknown')

        response = requests.delete(f"{BASE_URL}/WaterFacility/{osid}", headers=AUTH_HEADERS)

        if response.status_code == 200:
            print(f"Deleted: {geoCode} ({osid})")
            deleted += 1
        else:
            print(f"Failed to delete: {geoCode} ({osid})")
            failed += 1

    print(f"\nDeleted {deleted} facilities")
    if failed > 0:
        print(f"Failed {failed} facilities")
else:
    print("No water facilities to delete")

## Summary

This notebook demonstrated:
- Checking registry health
- Listing all water facilities
- Creating new water facilities with auto-generated wfId
- Reading specific facilities
- Updating facilities
- Searching facilities by type
- Bulk operations with wfId display
- Testing duplicate rejection (wfId uniqueness)
- Exporting to CSV
- Deleting all facilities

### wfId Generation

The `wfId` is automatically generated when creating a WaterFacility using the format:
```
WF-<COUNTY_ABBR>-<DISTRICT_ABBR>-<TYPE_CODE>-<HASH>
```

**Example:** `WF-NIM-BUU-TWB-CB37DF`

**Components:**
- `WF-` - Fixed prefix for Water Facility
- `COUNTY_ABBR` - First 3 alphanumeric characters of county (uppercase)
- `DISTRICT_ABBR` - First 3 alphanumeric characters of district (uppercase)
- `TYPE_CODE` - Water point type code:
  - `PDW` - Protected dug well
  - `UDW` - Unprotected dug well
  - `TWB` - Tube well or borehole
  - `PS` - Protected spring
  - `US` - Unprotected spring
  - `PWD` - Piped water into dwelling/plot/yard
  - `PTS` - Public tap/standpipe
  - `UEB` - Unequipped borehole
  - `RWH` - Rainwater (harvesting)
  - `SSD` - Sand/Sub-surface dam (with well or standpipe)
  - `OTH` - Other
- `HASH` - 6-character SHA-256 hash of normalized attributes

**Hash inputs (normalized to lowercase, trimmed):**
- geoCode
- waterPointType
- location.county
- location.district
- location.community

**Uniqueness:** The wfId is marked as a unique index field. Duplicate facilities (same core attributes) will be rejected.

### Water Facility Schema Fields

**Required Fields:**
- `geoCode` - Unique geographic code identifier
- `location` - Administrative location object
  - `county` - County name
  - `district` - District name
  - `community` - Community name
  - `coordinates` - Geographic coordinates (optional)
    - `lat` - Latitude
    - `lon` - Longitude
    - `elevation` - Elevation in meters
- `waterPointType` - Type of water point (enum)

**Auto-generated Fields:**
- `wfId` - System-generated unique ID (format: WF-XXX-XXX-TYPE-HASH)

**Optional Fields:**
- `extractionType` - Water extraction method (Manual, Electrical, Solar, Other)
- `extractionTypeOther` - Other extraction type specification
- `pumpType` - Type of pump installed (Afridev, Consallen, India Mark, Kardia, Rope pump, Vergnet, Other)
- `pumpTypeOther` - Other pump type specification
- `numTaps` - Number of taps
- `hasDepthInfo` - Whether depth information is available
- `depthMetres` - Depth in metres
- `installer` - Entity that installed (Government, NGO, Private, Other)
- `installerOther` - Other installer specification
- `owner` - Owner of the water point (Community, Private Individual, School, NGO, Health Facility, Other institution, CBO, Private, Unknown, Other)
- `funder` - Funding organization or entity
- `photoUrl` - URL to photo of the water point

### Water Point Types Supported
- Protected dug well (PDW)
- Unprotected dug well (UDW)
- Tube well or borehole (TWB)
- Protected spring (PS)
- Unprotected spring (US)
- Piped water into dwelling/plot/yard (PWD)
- Public tap/standpipe (PTS)
- Unequipped borehole (UEB)
- Rainwater (harvesting) (RWH)
- Sand/Sub-surface dam (with well or standpipe) (SSD)
- Other (OTH)

For more information, see the WaterFacility schema at `java/registry/src/main/resources/public/_schemas/WaterFacility.json`